##### 1、生成数据

In [6]:
import torch
import matplotlib.pyplot as plt
from torch.nn import functional as F   # 引入激活函数方法

—— unsqueeze(﹡,dim) 增加维度 ★

    dim=0时，在第0维上增加维度，将数据转换为[1,-1]类型
    dim=1时，在第1维上增加维度，将数据转换为[-1,1]类型

In [46]:
data = torch.linspace(-1,1,100)   # 一行数据
x = torch.unsqueeze(data, dim=1)   # 转换为一列数据
y = x.pow(2) 
print(data.size(),x.size())

torch.Size([100]) torch.Size([100, 1])


In [ ]:
plt.scatter(x.data, y.data, s=10, cmap='autumn')
plt.show()

##### 2、构建网络结构

—— torch.nn.Module：深度学习的基类
    
＿init＿(self, n_features, n_hidden, n_output)
        
        n_features：输入层神经元数（维度）
        n_hidden：隐层神经元数
        n_output：输出神经元数
        
—— torch.nn.Linear：线性映射

In [11]:
class SimpleNet(torch.nn.Module):
    # 构造方法
    def __init__(self, n_features, n_hidden, n_output): # 单隐含层网络，传入（1，10，1）
        super(SimpleNet, self).__init__()   # 对父类初始化
        self.hidden = torch.nn.Linear(n_features, n_hidden)   # 输入层->隐藏层  注意self.
        self.predict = torch.nn.Linear(n_hidden, n_output)   # 隐藏层->输出层
    
    # forward相当于对Module中的forward函数进行重构 ★
    def forward(self, x):
        hidden_result = self.hidden(x)   # 输入数据
        x = F.relu(hidden_result)   # ReLu激活函数
        x = self.predict(x)   # 输出数据
        return x

##### 3、设定网络参数
—— torch.optim.SGD：SGD梯度下降

    lr：学习率

—— torch.nn.MSELoss():MSE损失计算方法

In [49]:
mynet = SimpleNet(1, 10, 1)   # 实例化类，输入网络参数。有时也可以直接在网络结构的构造方法中直接指定网络参数，这样实例化时就用()即可
print(mynet.parameters)

loss_func = torch.nn.MSELoss()   # 计算损失值
optimizer = torch.optim.SGD(mynet.parameters(), lr=0.1)   # optimizer获取所有parameters的引用，每个parameter都包含梯度

<bound method Module.parameters of SimpleNet(
  (hidden): Linear(in_features=1, out_features=10, bias=True)
  (predict): Linear(in_features=10, out_features=1, bias=True)
)>


##### 4、模型训练

—— optimizer.zero_grad()：梯度清零

—— optimizer.step()等同于：

    linear.weight.data.sub_(0.01 * linear.weight.grad.data)
    linear.bias.data.sub_(0.01 * linear.bias.grad.data)

##### loss 与 optimizer的关联
1、梯度：

①、FP过程：对prediction和y之间进行比对（熵或者其他loss function），产生最初的梯度

②、BP过程：loss.backward()获得所有parameter的gradient，反向传播到整个网络的所有链路和节点

2、参数更新：

optimizer存了parameter的指针，step()根据这些parameter的gradient对parameter的值进行更新

这里不需要传入梯度，因为梯度的引用已经在其构造函数中传入的mynet.parameters()中包含了

In [22]:
for epoch in range(2000):   # 前后传播总次数
    optimizer.zero_grad()

    # forward + backward + optimize ★★
    pred = mynet(x)   # 自动调用forward方法
    loss = loss_func(pred, y)
    loss.backward()
    optimizer.step()   # 计算完一次反向梯度后，进行参数（w，b）迭代更新
    
    if(epoch%300 ==0): 
        print(loss.data)

tensor(0.0007)
tensor(0.0006)
tensor(0.0006)
tensor(0.0005)
tensor(0.0005)
tensor(0.0005)
tensor(0.0004)


##### 5、预测

In [18]:
test_data = torch.tensor([-1.0])
pred = mynet(test_data)   # 输出测试数据
print(pred.data)

tensor([0.9370])


## 关于forward的魔术方法：

mynet(x)实际上等价于mynet.forward(x)

等价的原因是因为 calss 中的__call__和__init__方法，如下：

In [39]:
# 例1
class A():
    def __call__(self):
        print('i can be called like a function')
 
a = A()
a()   # 调用魔法函数

i can be called like a function


In [37]:
# 例2
class A():
    def __call__(self, param):
        
        print('i can called like a function')
        print('传入参数的类型是：{}   值为： {}'.format(type(param), param))
 
        res = self.forward(param)   # 在__call__里调用其他的函数
        return res+1
 
    def forward(self, input_):
        print('forward 函数被调用了')
        print('in  forward, 传入参数类型是：{}  值为: {}'.format( type(input_), input_+1))
        return input_+1   # 执行到此返回2给res，然后再+1作为类调用魔法函数的返回值

a = A()   # 实例化时还是与普通实例化相同，不受影响
output_param = a(1)   # 调用魔法函数
print("对象a输出的参数是：", output_param)

i can called like a function
传入参数的类型是：<class 'int'>   值为： 1
forward 函数被调用了
in  forward, 传入参数类型是：<class 'int'>  值为: 2
对象a输出的参数是： 3
